In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd

import time

import pandas as pd

import re

In [2]:
import moviepy

In [3]:
# si da error unicode copias pegas el archivo entero a otro y a correr
df = pd.read_csv('data52kg.csv', index_col=0)

In [4]:
df = pd.read_csv('data66kg.csv', index_col=0)

In [5]:
# abrir Google Chrome
driver = webdriver.Chrome(r"C:\Users\manuel.lopez\Documents\chromedriver.exe")
try:
    driver.maximize_window()
except:
    pass

In [6]:
comp_types = [
    'Olympic Games',
    'World Championships',
    'Masters',
    'Grand Slam',
    'Grand Prix',
    'Continental Championships',
    'Continental Open'
]

In [7]:
def find_comp_type(comp):
    'Given string, identifies one of comp_types'
    comp_type = next((c for c in comp_types if c in comp), 'None')
    rest = comp.replace(comp_type, ' ').strip()

    return comp_type

def find_comp_rest(comp):
    'Given string, identifies one of comp_types'
    comp_type = next((c for c in comp_types if c in comp), 'None')
    rest = comp.replace(comp_type, ' ').strip()
    
    return rest

In [8]:
find_comp_type('World Championships Baku 2018')

'World Championships'

In [9]:
find_comp_rest('World Championships Baku 2018')

'Baku 2018'

In [10]:
df['comp_type'] = df.event.apply(find_comp_type)

In [11]:
df['comp_name'] = df.event.apply(find_comp_rest)

In [12]:
df.date = pd.to_datetime(df.date)

In [13]:
df.head()

,wins,local,opponent,local_points,opponent_points,duration,date,event,category,round,has_video,url_video,comp_type,comp_name
0,1,ABE Hifumi,SERIKZHANOV Yerlan,100,0,02:47,2018-09-20,World Championships Baku 2018,-66 kg,Final,True,https://judobase.ijf.org/#/competition/contest...,World Championships,Baku 2018
1,1,ABE Hifumi,AN Baul,10,0,02:38,2018-09-20,World Championships Baku 2018,-66 kg,Semi-Final,True,https://judobase.ijf.org/#/competition/contest...,World Championships,Baku 2018
2,1,ABE Hifumi,ZANTARAIA Georgii,100,0,03:10,2018-09-20,World Championships Baku 2018,-66 kg,Quarter-Final,True,https://judobase.ijf.org/#/competition/contest...,World Championships,Baku 2018
3,1,ABE Hifumi,VIERU Denis,10,0,00:00,2018-09-20,World Championships Baku 2018,-66 kg,Round 4,True,https://judobase.ijf.org/#/competition/contest...,World Championships,Baku 2018
4,1,ABE Hifumi,MEDVES Matteo,100,0,02:57,2018-09-20,World Championships Baku 2018,-66 kg,Round 3,True,https://judobase.ijf.org/#/competition/contest...,World Championships,Baku 2018


### Dada url video judobase, extraer info y acceder al youtube

In [14]:
# elige una random y ábrela
while True:
    url = df.sample(1).iloc[0].url_video
    if not pd.isnull(url):
        break

In [15]:
driver.get(url)
time.sleep(.5)

### Extraer info

In [16]:
# extract names
local, opponent = map(lambda x: x.text, driver.find_elements_by_class_name('col-xs-6'))
local, opponent = map(lambda x: x[:x.find('\n')], [local, opponent])

In [17]:
local

'MATEO Wander'

In [18]:
opponent

'ALTYNBEK UULU Almaz'

In [19]:
# tabla con puntuaciones etiquetadas en tiempo
tabla_timed_points = driver.find_elements_by_tag_name('tbody')[1]

In [20]:
filas = tabla_timed_points.find_elements_by_tag_name('tr')

In [21]:
nrows = len(filas[0].find_elements_by_tag_name('td'))
nrows

3

In [22]:
if nrows == 2:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

else:
    print(f'{nrows} rows!')

3 rows!


In [23]:
youtube_frame = driver.find_element_by_class_name('js-media')

In [24]:
driver.switch_to.frame(youtube_frame)

In [25]:
url_youtube = driver.find_element_by_class_name('ytp-title-link').get_attribute('href')

url_youtube

'https://www.youtube.com/watch?v=TgFQMxD4aNs'

In [26]:
seconds = 22

In [27]:
url_youtube_time = url_youtube + '&t=' + str(seconds)
url_youtube_time

'https://www.youtube.com/watch?v=TgFQMxD4aNs&t=22'

In [28]:
driver.get(url_youtube_time)

### Youtube downloads

In [29]:
from pytube import YouTube

In [30]:
y = YouTube(url_youtube_time)

In [31]:
[d for d in dir(y) if d[0]!='_']

['age_restricted',
 'caption_tracks',
 'captions',
 'description',
 'embed_html',
 'embed_url',
 'fmt_streams',
 'init',
 'initialize_caption_objects',
 'initialize_stream_objects',
 'js',
 'js_url',
 'length',
 'player_config_args',
 'prefetch',
 'prefetch_init',
 'rating',
 'register_on_complete_callback',
 'register_on_progress_callback',
 'stream_monostate',
 'streams',
 'thumbnail_url',
 'title',
 'vid_descr',
 'vid_info',
 'vid_info_url',
 'video_id',
 'views',
 'watch_html',
 'watch_url']

In [32]:
video = y.streams.filter(res='720p', mime_type='video/mp4').first()

In [33]:
assert video is not None, 'No video'

In [34]:
video.download(filename='judo720p3')

'C:\\Users\\manuel.lopez\\Documents\\scrape\\0e6a72df62d07aa6cc043c46d63ad5e7\\judo720p3.mp4'

### Edición con moviepy

In [1]:
3

3

In [2]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_videoclips

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# Load myHolidays.mp4 and select the subclip 00:00:50 - 00:00:60
clip = VideoFileClip('./judo720p3.mp4')

In [4]:
clip.duration

416.82

In [5]:
cortes = [clip.subclip(t, t+5) for t in range(10, 40, 10)]

In [6]:
len(cortes)

3

In [31]:
text0.set_position??

In [34]:
text0 = TextClip("Primer chunk", fontsize=70, color='red').set_position(('right', 'top'), relative=True).set_duration(5)
text1 = TextClip("Segundo chunk", fontsize=70, color='blue').set_position((0.4, 0), relative=True).set_duration(5)
text2 = TextClip("Tercer chunk", fontsize=70, color='green').set_position((0.8, 0), relative=True).set_duration(5)

textos = (text0, text1, text2)

In [35]:
cortes_texto = [CompositeVideoClip([corte, texto]) for corte, texto in zip(cortes, textos)]

In [36]:
cortes_texto[0].duration

5

In [37]:
final_clip = concatenate_videoclips(cortes_texto)

In [39]:
final_clip.write_videofile("my_concatenation.mp4")

[MoviePy] >>>> Building video my_concatenation.mp4
[MoviePy] Writing audio in my_concatenationTEMP_MPY_wvf_snd.mp3


100%|███████████████████████████████████████████████████████████████████████████████| 331/331 [00:01<00:00, 304.79it/s]


[MoviePy] Done.
[MoviePy] Writing video my_concatenation.mp4


100%|███████████████████████████████████████████████████████████████████████████████▊| 375/376 [00:20<00:00, 18.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: my_concatenation.mp4 

